
$\qquad$ $\qquad$$\qquad$  **TDA231/DIT370 Discrete Optimization: Home Assignment 1 -- Modelling and Solving with LPs** <br />
$\qquad$ $\qquad$$\qquad$                   **Grader: Divya** <br />
$\qquad$ $\qquad$$\qquad$                     **Due Date: 27th January** <br />
$\qquad$ $\qquad$$\qquad$                   **Submitted by:   Julia Molin, 950322-4124, mojulia@student.chalmers.se** <br />$\qquad$ $\qquad$$\qquad$$\qquad$$\qquad$$\quad$ **Ludwig Hultqvist, 970917-0071, ludhul@student.chalmers.se**<br />


---


General guidelines:
*   All solutions to theoretical and pratical problems must be submitted in this ipynb notebook, and equations wherever required, should be formatted using LaTeX math-mode.
*   All discussion regarding practical problems, along with solutions and plots should be specified in this notebook. All plots/results should be visible such that the notebook do not have to be run. But the code in the notebook should reproduce the plots/results if we choose to do so.
*   Your name, personal number and email address should be specified above.
*   All tables and other additional information should be included in this notebook.
*   Before submitting, make sure that your code can run on another computer. That all plots can show on another computer including all your writing. It is good to check if your code can run here: https://colab.research.google.com.


# Problem 1.

Consider the following LP problem:


\begin{array}
\mathcal{max}\quad 4x_1-2x_2+5x_3+6x_4+7x_5\\
\textrm{s.t} \\
2x_1 + 2x_2 - 4x_3 + 4x_4 + 8x_5 \leq 6\\
2x_1 + x_2 - 2x_3 - x_4 - 3x_5 \geq -1\\
5x_1 - 2x_2 + 4x_3 + 4x_4 + 2x_5 = 5\\
2x_1 - 2x_2 + 5x_3 + 3x_4 + x_5 \leq 4\\
\vec x \geq \vec 0
\end{array}

* (10 points) Use CVXOPT to solve the LP above. Submit your code, and print the solution vector and objective value.

### Solution 1 - code

In [37]:
import cvxpy as cp
import numpy as np

A = np.array([
             [2,2,-4,4,8], 
             [-2,-1,2,1,3],
             [5,-2,4,4,2],
             [-5,2,-4,-4,-2],
             [2,-2,5,3,1]
             ])

b = np.array([6,1,5,-5,4])

c = np.array([4,-2,5,6,7])

# Construct the problem.
x = cp.Variable(5, nonneg=True)
constraints = [A @ x <= b, 0 <= x]
objective = cp.Maximize(c.T@x) 
prob = cp.Problem(objective, constraints)

# Solve
result = prob.solve()
print("x values: ", x.value)
print("optimal solution: ", result)

#print(constraints[0].dual_value)


x values:  [6.27118644e-01 2.81355932e+00 1.54237288e+00 1.67556947e-11
 6.61016949e-01]
optimal solution:  9.220338982799781


# Problem 2.

There are 4 space colonies, each of which  requires a certain number of plasma conduits. There are 3 starbases in the vicinity. Each of them has total number of conduits they can spare and supply to the colonies. For each pair of starbase and colony, there is an associated cost for sending a cargo ship  (each of which carries one plasma conduit), as shown in the table below:


\begin{array}{l|c|c|c|c|c} 
      & Triacus & New Berlin  & Strnad  & Vega  & supply\\ \hline
 Farpoint &   6 &  9 & 10 & 8 & 35\\
 Yorktown &  9 & 5 & 16 & 14 & 40\\
 Earhart & 12 &  7 & 13 & 9 & 50\\ \hline
    demand & 20 &30&30&45& \left(\sum=125\right) \\ 
\end{array}

Your goal is to supply the colonies the plasma conduits they need, at minimum cost.


* (5 points) Consider the general *transportation problem*: where there are $\bf{n}$ colonies and $\bf{m}$ bases and the costs are given by a $\bf{m} \times \bf{n}$ matrix $\mathcal{C}$, demand and supply are given by arrays $\bf{d}$ and $\bf{s}$ respectively. Formulate a LP to solve the problem.

* (3 points) Code the LP in CVXOPT, input the data for the space colonies manually and use CVXOPT to solve the LP. Submit your code and write down the solution and objective.

* (2 points) Use CVXOPT to show what  the  effect  on the model and the optimal solution would be if each of the starbases could supply five more conduits.

### Solution 2.1 
We define $c_{ij}$ as the cost to transport from starbase i to colony j, 
and $x_{ij}$ as the number of conduits which are transported from i to j.
Further we define $d_j$ as the demand of colony j, and $s_i$ as the supply from starbase i.

We want to minimize the sum of all costs.
We cannot send more conduits from a starbase than its supply, and each colony must get at least its demand. This gives us the following LP:


##### Objective function: 
$ \min \sum_{i}{c_{ij} x_{ij}} $

##### Constraints:
$ \sum_{i}{x_{ij}} \geq d_j \quad \forall j$ 

$ \sum_{j}{x_{ij}} \leq s_i \quad \forall i$

$x_{ij} \geq 0 \quad \forall ij$


### Solution 2.2 - code

To achive the objective function, we take the diagonal sum of the dot product of the transposed cost matrix and our variable matrix (X). Thus, we only need one cost matrix and one variable matrix.

To reduce the number of explicit constraints, we keep supplies and demands as vectors. Further we use the help-vectors ei and ej to simplify the constraints.

##### Question to TA:
When we don't use the param "nonneg=True" in variable X, the LP-solver gives a negative value for one of the x values, even though we define a constraint [x >= 0]. Do you know why? 

In [38]:
supplies = np.array([35,40,50])
demands = np.array([20,30,30,45])

costs = np.array([[6, 9, 10, 8],
                  [9, 5, 16, 14],
                  [12, 7, 13, 9]])

ei = np.ones(supplies.shape) #[1,1,1]
ej = np.ones(demands.shape)  #[1,1,1,1]

X = cp.Variable(costs.shape,"X", nonneg=True)

objective = cp.Minimize(cp.trace(costs.T @ X)) #sum of all cost_ij * x_ij
constraints = [X.T@ei >= demands, 
               X@ej <= supplies, 
               X>=0]

prob = cp.Problem(objective, constraints)

prob.solve()
print("X-values: ", X.value)
print("status:", prob.status)
print("objective:",prob.value)

X-values:  [[9.99999999e+00 1.45595158e-09 2.50000000e+01 5.63585061e-09]
 [1.00000000e+01 3.00000000e+01 6.43021238e-09 3.08607353e-09]
 [1.13775877e-08 1.06620583e-08 4.99999999e+00 4.50000000e+01]]
status: optimal
objective: 1020.0000000943861


### Solution 2.3 - more supply
When we increased the supply, we observed that the optimal cost decreased from 1020.0 to 929.9

In [31]:
supplies += 5 #increase all entries with 5

X2 = cp.Variable(costs.shape,"X", nonneg=True)
objective2 = cp.Minimize(cp.trace(costs.T @ X2)) #sum of all cost_ij * x_ij

constraints2 = [X2.T@ei >= demands, 
               X@ej <= supplies, 
               X>=0]

prob2 = cp.Problem(objective2, constraints2)

prob2.solve()
print(X2.value)
print("status:",prob2.status)
print("objective:",prob2.value)


[[2.00000000e+01 0.00000000e+00 3.00000000e+01 4.50000000e+01]
 [0.00000000e+00 3.00000000e+01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.12010803e-10 4.91726278e-11 1.17511612e-09]]
status: optimal
objective: 929.9999999901953
